# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = "output_data/cities.csv"
cities_df = pd.read_csv(cities)
del cities_df["Date"]
cities_df.head()

,City,Temperature,Country,Cloudiness,Humidity,Latitude,Longitude,Wind Speed
0,Ushuaia,2.00,AR,20,80,-54.80,-68.30,9.80
1,Bethel,21.11,US,1,64,41.37,-73.41,1.50
2,George Town,29.00,MY,20,88,5.41,100.34,1.50
3,Hermanus,12.22,ZA,43,87,-34.42,19.23,7.15
4,Albany,22.22,US,0,84,42.60,-73.97,1.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(layout={
        'width': '1000px',
        'height': '700px',
        'padding': '7px',
        'border': '2px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='700px', padding='7px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df = cities_df.loc[(cities_df["Wind Speed"] <= 5) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Temperature"] >= 24)].dropna()
new_df

,City,Temperature,Country,Cloudiness,Humidity,Latitude,Longitude,Wind Speed
63,Datong,26.24,CN,0,28,40.09,113.29,0.93
107,Laguna,32.78,US,0,45,38.42,-121.42,2.24
208,Tabas,29.00,IR,0,20,33.60,56.92,2.10
253,Intipucá,26.41,SV,0,73,13.20,-88.05,0.36
307,Adrar,35.82,MR,0,14,20.50,-10.07,3.06
318,Kunda,30.83,IN,0,62,25.72,81.52,4.74
342,Dul'durga,24.77,RU,0,47,50.68,113.59,1.00
353,Tessalit,33.27,ML,0,11,20.20,1.01,4.18
365,Zhangjiakou,26.61,CN,0,33,40.81,114.88,1.39
375,Chapada dos Guimarães,25.00,BR,0,69,-15.46,-55.75,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = new_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Temperature,Country,Cloudiness,Humidity,Latitude,Longitude,Wind Speed,Hotel Name
63,Datong,26.24,CN,0,28,40.09,113.29,0.93,
107,Laguna,32.78,US,0,45,38.42,-121.42,2.24,
208,Tabas,29.00,IR,0,20,33.60,56.92,2.10,
253,Intipucá,26.41,SV,0,73,13.20,-88.05,0.36,
307,Adrar,35.82,MR,0,14,20.50,-10.07,3.06,
318,Kunda,30.83,IN,0,62,25.72,81.52,4.74,
342,Dul'durga,24.77,RU,0,47,50.68,113.59,1.00,
353,Tessalit,33.27,ML,0,11,20.20,1.01,4.18,
365,Zhangjiakou,26.61,CN,0,33,40.81,114.88,1.39,
375,Chapada dos Guimarães,25.00,BR,0,69,-15.46,-55.75,3.60,


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows():
# get city name, lat, lnt from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except :
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 63: Datong.
Closest hotel in Datong is Datong Weidu International Hotel.
------------
Retrieving Results for Index 107: Laguna.
Closest hotel in Laguna is Hilton Garden Inn Sacramento Elk Grove.
------------
Retrieving Results for Index 208: Tabas.
Closest hotel in Tabas is Amir hotel.
------------
Retrieving Results for Index 253: Intipucá.
Closest hotel in Intipucá is Tropiclub Playa El Cuco.
------------
Retrieving Results for Index 307: Adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 318: Kunda.
Closest hotel in Kunda is Jaishwaal Guest House.
------------
Retrieving Results for Index 342: Dul'durga.
Closest hotel in Dul'durga is Gostinitsa.
------------
Retrieving Results for Index 353: Tessalit.
Missing field/result... skipping.
------------
Retrieving Results for Index 365: Zhangjiakou.
Closest hotel in Zhangjiakou is Zhangjiakou International Hotel.
------------
Retrieving Results for Index 375: Chapada dos Guimarã

In [8]:
hotel_df

,City,Temperature,Country,Cloudiness,Humidity,Latitude,Longitude,Wind Speed,Hotel Name
63,Datong,26.24,CN,0,28,40.09,113.29,0.93,Datong Weidu International Hotel
107,Laguna,32.78,US,0,45,38.42,-121.42,2.24,Hilton Garden Inn Sacramento Elk Grove
208,Tabas,29.00,IR,0,20,33.60,56.92,2.10,Amir hotel
253,Intipucá,26.41,SV,0,73,13.20,-88.05,0.36,Tropiclub Playa El Cuco
307,Adrar,35.82,MR,0,14,20.50,-10.07,3.06,
318,Kunda,30.83,IN,0,62,25.72,81.52,4.74,Jaishwaal Guest House
342,Dul'durga,24.77,RU,0,47,50.68,113.59,1.00,Gostinitsa
353,Tessalit,33.27,ML,0,11,20.20,1.01,4.18,
365,Zhangjiakou,26.61,CN,0,33,40.81,114.88,1.39,Zhangjiakou International Hotel
375,Chapada dos Guimarães,25.00,BR,0,69,-15.46,-55.75,3.60,Hotel Turismo


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='2px solid black', height='700px', padding='7px', width='1000px'))